In [34]:
import sys

this_python = sys.version_info[:2]
min_version = (3, 9)
if this_python < min_version:
    message_parts = [
        "This script does not work on Python {}.{}.".format(*this_python),
        "The minimum supported Python version is {}.{}.".format(*min_version),
        "Please use https://bootstrap.pypa.io/pip/{}.{}/get-pip.py instead.".format(*this_python),
    ]
    print("ERROR: " + " ".join(message_parts))
    sys.exit(1)


import os.path
import pkgutil
import shutil
import tempfile
import argparse
import importlib
from base64 import b85decode


def include_setuptools(args):
    """
    Install setuptools only if absent, not excluded and when using Python <3.12.
    """
    cli = not args.no_setuptools
    env = not os.environ.get("PIP_NO_SETUPTOOLS")
    absent = not importlib.util.find_spec("setuptools")
    python_lt_3_12 = this_python < (3, 12)
    return cli and env and absent and python_lt_3_12


def include_wheel(args):
    """
    Install wheel only if absent, not excluded and when using Python <3.12.
    """
    cli = not args.no_wheel
    env = not os.environ.get("PIP_NO_WHEEL")
    absent = not importlib.util.find_spec("wheel")
    python_lt_3_12 = this_python < (3, 12)
    return cli and env and absent and python_lt_3_12


def determine_pip_install_arguments():
    pre_parser = argparse.ArgumentParser()
    pre_parser.add_argument("--no-setuptools", action="store_true")
    pre_parser.add_argument("--no-wheel", action="store_true")
    pre, args = pre_parser.parse_known_args()

    args.append("pip")

    if include_setuptools(pre):
        args.append("setuptools")

    if include_wheel(pre):
        args.append("wheel")

    return ["install", "--upgrade", "--force-reinstall"] + args


def monkeypatch_for_cert(tmpdir):
    """Patches `pip install` to provide default certificate with the lowest priority.

    This ensures that the bundled certificates are used unless the user specifies a
    custom cert via any of pip's option passing mechanisms (config, env-var, CLI).

    A monkeypatch is the easiest way to achieve this, without messing too much with
    the rest of pip's internals.
    """
    from pip._internal.commands.install import InstallCommand

    # We want to be using the internal certificates.
    cert_path = os.path.join(tmpdir, "cacert.pem")
    with open(cert_path, "wb") as cert:
        cert.write(pkgutil.get_data("pip._vendor.certifi", "cacert.pem"))

    install_parse_args = InstallCommand.parse_args

    def cert_parse_args(self, args):
        if not self.parser.get_default_values().cert:
            # There are no user provided cert -- force use of bundled cert
            self.parser.defaults["cert"] = cert_path  # calculated above
        return install_parse_args(self, args)

    InstallCommand.parse_args = cert_parse_args


def bootstrap(tmpdir):
    monkeypatch_for_cert(tmpdir)

    # Execute the included pip and use it to install the latest pip and
    # any user-requested packages from PyPI.
    from pip._internal.cli.main import main as pip_entry_point
    args = determine_pip_install_arguments()
    sys.exit(pip_entry_point(args))


def main():
    tmpdir = None
    try:
        # Create a temporary working directory
        tmpdir = tempfile.mkdtemp()

        # Unpack the zipfile into the temporary directory
        pip_zip = os.path.join(tmpdir, "pip.zip")
        with open(pip_zip, "wb") as fp:
            fp.write(b85decode(DATA.replace(b"\n", b"")))

        # Add the zipfile to sys.path so that we can import it
        sys.path.insert(0, pip_zip)

        # Run the bootstrap
        bootstrap(tmpdir=tmpdir)
    finally:
        # Clean up our temporary working directory
        if tmpdir:
            shutil.rmtree(tmpdir, ignore_errors=True)


In [35]:
import csv
import os
import re
import threading
from datetime import datetime
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import random
from tkinter import ttk, messagebox, Toplevel
from tkinter import simpledialog

In [36]:
from tkcalendar import DateEntry
print("✅ tkcalendar is working!")

✅ tkcalendar is working!


In [37]:
from fpdf import FPDF

In [38]:
# Constants
CSV_FILE = "hiker_data.csv"
FUN_FACTS_FILE = "fun_facts.txt"
REMINDER_DAYS = 3

In [39]:
def admin_dashboard(parent):
    dash = Toplevel(parent)
    dash.title("Admin Dashboard")
    ...


In [ ]:
# Hiker Class
class Hiker:
    def __init__(self, name, contact, hike, hike_date, payment, gear, date=None):
        self.name = name
        self.contact = contact
        self.hike = hike
        self.hike_date = hike_date
        self.payment = payment
        self.gear = gear
        self.date = date or datetime.today().strftime('%Y-%m-%d')

    def to_list(self):
        return [self.name, self.contact, self.hike, self.hike_date, self.payment, self.gear, self.date]

# Setup CSV File
if not os.path.exists(CSV_FILE):
    with open(CSV_FILE, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Name", "Contact", "Hike", "Hike Date", "Payment Status", "Gear Rental", "Date Registered"])

# Validators
def is_valid_contact(contact):
    phone_pattern = r"^07\d{8}$"
    email_pattern = r"^[\w.-]+@[\w.-]+\.\w{2,}$"
    return re.match(phone_pattern, contact) or re.match(email_pattern)

# Save
def save_registration(hiker):
    with open(CSV_FILE, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(hiker.to_list())

# Receipt
def generate_receipt(hiker):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt="Hiker Registration Receipt", ln=True, align='C')
    for key, value in zip(["Name", "Contact", "Hike", "Hike Date", "Payment", "Gear", "Date"], hiker.to_list()):
        pdf.cell(200, 10, txt=f"{key}: {value}", ln=True)
    file_path = f"receipt_{hiker.name.replace(' ', '_')}.pdf"
    pdf.output(file_path)
    return file_path

# Reminder
def schedule_reminder(hiker):
    def remind():
        messagebox.showinfo("Reminder", f"Reminder: {hiker.name} has a pending payment.")
    delay = REMINDER_DAYS * 86400
    threading.Timer(delay, remind).start()

# Submit Logic
def submit():
    name = name.get()
    contact = contact.get()
    hike = hike.get()
    hike_date = hike_date.get()
    payment = payment_var.get()
    gear = ", ".join([g.get() for g in gear_vars if g.get()])

    if not name or not contact or not hike or not hike_date or not payment:
        messagebox.showwarning("Input Error", "Please fill all required fields.")
        return

    if not is_valid_contact(contact):
        messagebox.showerror("Invalid Contact", "Enter a valid phone number or email.")
        return

    hiker = Hiker(name, contact, hike, hike_date, payment, gear)
    save_registration(hiker)
    receipt_path = generate_receipt(hiker)

    msg = f"{name} registered successfully.\nReceipt saved to {receipt_path}."
    if payment.lower() == "partial":
        schedule_reminder(hiker)
        msg += f"\nReminder scheduled after {REMINDER_DAYS} days."

    #fun_fact = random.choice(FUN_FACTS)
    #messagebox.showinfo("Success", f"{msg}\n\n💡 Fun Fact: {fun_fact}")

    for entry in [name_entry, contact_entry, hike_entry]:
        entry.delete(0, tk.END)
    payment_var.set(None)
    for g in gear_vars:
        g.set("")
 

In [ ]:
#HIKER GUI

# GUI
root = tk.Tk()
root.title("Hiker Registration Tracker")
root.geometry("500x500")

tk.Label(root, text="Name").pack()
name_entry = tk.Entry(root)
name_entry.pack()

tk.Label(root, text="Contact (Phone or Email)").pack()
contact_entry = tk.Entry(root)
contact_entry.pack()

tk.Label(root, text="Hike Name").pack()
hike_entry = tk.Entry(root)
hike_entry.pack()

tk.Label(root, text="Hike Date").pack()
hike_date_entry = DateEntry(root, date_pattern='yyyy-mm-dd')
hike_date_entry.pack()

tk.Label(root, text="Payment Status").pack()
payment_var = tk.StringVar()
tk.Radiobutton(root, text="Full", variable=payment_var, value="Full").pack()
tk.Radiobutton(root, text="Partial", variable=payment_var, value="Partial").pack()

tk.Label(root, text="Gear Rental (Optional)").pack()
gear_options = ["Tent", "Sleeping Bag", "Hiking Pole"]
gear_vars = [tk.StringVar() for _ in gear_options]
for opt, var in zip(gear_options, gear_vars):
    tk.Checkbutton(root, text=opt, variable=var, onvalue=opt, offvalue="").pack(anchor='w')

tk.Button(root, text="Submit Registration", command=submit).pack(pady=20)

root.mainloop()

In [41]:
# GUI
root = tk.Tk()
root.title("Hiker Registration Tracker")
root.geometry("500x500")

tk.Label(root, text="Name").pack()
name_entry = tk.Entry(root)
name_entry.pack()

tk.Label(root, text="Contact (Phone or Email)").pack()
contact_entry = tk.Entry(root)
contact_entry.pack()

tk.Label(root, text="Hike Name").pack()
hike_entry = tk.Entry(root)
hike_entry.pack()

tk.Label(root, text="Hike Date").pack()
hike_date_entry = DateEntry(root, date_pattern='yyyy-mm-dd')
hike_date_entry.pack()

tk.Label(root, text="Payment Status").pack()
payment_var = tk.StringVar()
tk.Radiobutton(root, text="Full", variable=payment_var, value="Full").pack()
tk.Radiobutton(root, text="Partial", variable=payment_var, value="Partial").pack()

tk.Label(root, text="Gear Rental (Optional)").pack()
gear_options = ["Tent", "Sleeping Bag", "Hiking Pole"]
gear_vars = [tk.StringVar() for _ in gear_options]
for opt, var in zip(gear_options, gear_vars):
    tk.Checkbutton(root, text=opt, variable=var, onvalue=opt, offvalue="").pack(anchor='w')

tk.Button(root, text="Submit Registration", command=submit).pack(pady=20)

root.mainloop()
 

In [48]:
#Admin Class

class Admin:
    file_path = "admins.csv"
    headers = ["ID", "Name", "Email", "Phone", "Role"]


    @classmethod
    def load_data(cls):
        if os.path.exists(cls.file_path):
            return pd.read_csv(cls.file_path)
        else:
            return pd.DataFrame(columns=cls.headers)

    @classmethod
    def save_data(cls, df):
        df.to_csv(cls.file_path, index=False)

    @classmethod
    def add_record(cls, record):
        df = cls.load_data()
        new_id = df["ID"].max() + 1 if not df.empty else 1
        record = [new_id] + record
        df.loc[len(df)] = record
        cls.save_data(df)

    @classmethod
    def delete_record(cls, record_id):
        df = cls.load_data()
        df = df[df["ID"] != record_id]
        cls.save_data(df)

    @classmethod
    def update_record(cls, record_id, updated_record):
        df = cls.load_data()
        df.loc[df["ID"] == record_id, cls.headers[1:]] = updated_record[1:]  # skip ID
        cls.save_data(df)

    @classmethod
    def search_records(cls, term):
        df = cls.load_data()
        return df[df.apply(lambda row: row.astype(str).str.contains(term, case=False).any(), axis=1)]

# ---------- Admin Dashboard GUI ----------
def admin_dashboard():
    root = tk.Tk()
    root.title("Admin Dashboard")
    root.geometry("1000x600")

    search_var = tk.StringVar()

    def load_treeview(df):
        for row in tree.get_children():
            tree.delete(row)
        for index, row in df.iterrows():
            tree.insert('', 'end', iid=row["ID"], values=list(row))

    def search():
        term = search_var.get()
        result = Admin.search_records(term)
        load_treeview(result)

    def delete_selected():
        selected = tree.selection()
        if not selected:
            messagebox.showwarning("Select Record", "No record selected.")
            return
        for record_id in selected:
            Admin.delete_record(int(record_id))
        load_treeview(Admin.load_data())

    def edit_selected():
        selected = tree.selection()
        if not selected:
            messagebox.showwarning("Select Record", "No record selected.")
            return
        record_id = int(selected[0])
        df = Admin.load_data()
        record = df[df["ID"] == record_id].values[0]

        edit_win = Toplevel(root)
        edit_win.title("Edit Admin Record")

        entries = []
        for i, field in enumerate(Admin.headers):
            tk.Label(edit_win, text=field).grid(row=i, column=0)
            entry = tk.Entry(edit_win, width=50)
            entry.insert(0, record[i])
            entry.grid(row=i, column=1)
            if field == "ID":
                entry.config(state='disabled')
            entries.append(entry)

        def save_changes():
            updated = [entry.get() for entry in entries]
            Admin.update_record(record_id, updated)
            edit_win.destroy()
            load_treeview(Admin.load_data())

        tk.Button(edit_win, text="Save", command=save_changes).grid(row=len(entries), column=1)

    def add_new():
        add_win = Toplevel(root)
        add_win.title("Add New Admin")

        entries = []
        for i, field in enumerate(Admin.headers[1:]):  # Skip ID for new
            tk.Label(add_win, text=field).grid(row=i, column=0)
            entry = tk.Entry(add_win, width=50)
            entry.grid(row=i, column=1)
            entries.append(entry)

        def save_new():
            new_data = [entry.get() for entry in entries]
            if any(val == "" for val in new_data):
                messagebox.showerror("Error", "Please fill all fields.")
                return
            Admin.add_record(new_data)
            add_win.destroy()
            load_treeview(Admin.load_data())

        tk.Button(add_win, text="Save", command=save_new).grid(row=len(entries), column=1)

def admin_login():
    password = simpledialog.askstring("Admin Login", "Enter admin password:", show='*')
    if password == ADMIN_PASSWORD:
        admin_dashboard(root)  # root is your main Tk instance
    else:
        messagebox.showerror("Error", "Incorrect password.")

    # GUI Widgets
    tk.Label(root, text="Search:").pack(pady=5)
    tk.Entry(root, textvariable=search_var).pack(pady=5)
    tk.Button(root, text="Search", command=search).pack()

    columns = Admin.headers
    tree = ttk.Treeview(root, columns=columns, show='headings')
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=150)
    tree.pack(pady=10, fill='both', expand=True)

    btn_frame = tk.Frame(root)
    btn_frame.pack(pady=10)

    tk.Button(btn_frame, text="Add New Admin", command=add_new).grid(row=0, column=0, padx=10)
    tk.Button(btn_frame, text="Edit Selected", command=edit_selected).grid(row=0, column=1, padx=10)
    tk.Button(btn_frame, text="Delete Selected", command=delete_selected).grid(row=0, column=2, padx=10)
    tk.Button(btn_frame, text="Refresh", command=lambda: load_treeview(Admin.load_data())).grid(row=0, column=3, padx=10)

    load_treeview(Admin.load_data())
    root.mainloop()

# ---------- Launch the App ----------
if __name__ == "__main__":
    admin_dashboard()


In [49]:

root = tk.Tk()

from tkinter import Toplevel
from tkinter import ttk

def admin_dashboard(parent):
    dash = Toplevel(parent)
    dash.title("Admin Dashboard")
    dash.geometry("1000x600")

    search_var = tk.StringVar()

    def load_treeview(df):
        for row in tree.get_children():
            tree.delete(row)
        for _, row in df.iterrows():
            tree.insert('', 'end', iid=row["ID"], values=list(row))

    def search():
        term = search_var.get()
        result = Admin.search_records(term)
        load_treeview(result)

    def delete_selected():
        selected = tree.selection()
        if not selected:
            messagebox.showwarning("Select Record", "No record selected.")
            return
        for record_id in selected:
            Admin.delete_record(int(record_id))
        load_treeview(Admin.load_data())

    def edit_selected():
        selected = tree.selection()
        if not selected:
            messagebox.showwarning("Select Record", "No record selected.")
            return
        record_id = int(selected[0])
        df = Admin.load_data()
        record = df[df["ID"] == record_id].values[0]

        edit_win = Toplevel(dash)
        edit_win.title("Edit Admin Record")

        entries = []
        for i, field in enumerate(Admin.headers):
            tk.Label(edit_win, text=field).grid(row=i, column=0)
            entry = tk.Entry(edit_win, width=50)
            entry.insert(0, record[i])
            entry.grid(row=i, column=1)
            if field == "ID":
                entry.config(state='disabled')
            entries.append(entry)

        def save_changes():
            updated = [entry.get() for entry in entries]
            Admin.update_record(record_id, updated)
            edit_win.destroy()
            load_treeview(Admin.load_data())

        tk.Button(edit_win, text="Save", command=save_changes).grid(row=len(entries), column=1)

    def add_new():
        add_win = Toplevel(dash)
        add_win.title("Add New Admin")

        entries = []
        for i, field in enumerate(Admin.headers[1:]):  # Skip ID
            tk.Label(add_win, text=field).grid(row=i, column=0)
            entry = tk.Entry(add_win, width=50)
            entry.grid(row=i, column=1)
            entries.append(entry)

        def save_new():
            new_data = [entry.get() for entry in entries]
            if any(val == "" for val in new_data):
                messagebox.showerror("Error", "Please fill all fields.")
                return
            Admin.add_record(new_data)
            add_win.destroy()
            load_treeview(Admin.load_data())

        tk.Button(add_win, text="Save", command=save_new).grid(row=len(entries), column=1)

    # GUI layout
    tk.Label(dash, text="Search:").pack(pady=5)
    tk.Entry(dash, textvariable=search_var).pack(pady=5)
    tk.Button(dash, text="Search", command=search).pack()

    columns = Admin.headers
    tree = ttk.Treeview(dash, columns=columns, show='headings')
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=150)
    tree.pack(pady=10, fill='both', expand=True)

    btn_frame = tk.Frame(dash)
    btn_frame.pack(pady=10)

    tk.Button(btn_frame, text="Add New Admin", command=add_new).grid(row=0, column=0, padx=10)
    tk.Button(btn_frame, text="Edit Selected", command=edit_selected).grid(row=0, column=1, padx=10)
    tk.Button(btn_frame, text="Delete Selected", command=delete_selected).grid(row=0, column=2, padx=10)
    tk.Button(btn_frame, text="Refresh", command=lambda: load_treeview(Admin.load_data())).grid(row=0, column=3, padx=10)

    load_treeview(Admin.load_data())


In [ ]:
# Admin Class 
class Admin:
    file_path = "admins.csv"
    headers = ["ID", "Name", "Email", "Phone", "Role"]

    @classmethod
    def load_data(cls):
        if os.path.exists(cls.file_path):
            return pd.read_csv(cls.file_path)
        else:
            return pd.DataFrame(columns=cls.headers)

    @classmethod
    def save_data(cls, df):
        df.to_csv(cls.file_path, index=False)

    @classmethod
    def add_record(cls, record):
        df = cls.load_data()
        new_id = df["ID"].max() + 1 if not df.empty else 1
        record = [new_id] + record
        df.loc[len(df)] = record
        cls.save_data(df)

    @classmethod
    def delete_record(cls, record_id):
        df = cls.load_data()
        df = df[df["ID"] != record_id]
        cls.save_data(df)

    @classmethod
    def update_record(cls, record_id, updated_record):
        df = cls.load_data()
        df.loc[df["ID"] == record_id, cls.headers[1:]] = updated_record[1:]  # Skip ID
        cls.save_data(df)

    @classmethod
    def search_records(cls, term):
        df = cls.load_data()
        return df[df.apply(lambda row: row.astype(str).str.contains(term, case=False).any(), axis=1)]

# Admin Dashboard
def open_admin_dashboard():
    dash = Toplevel()
    dash.title("Admin Dashboard")
    dash.geometry("600x600")

    search_var = tk.StringVar()

    def load_treeview(df):
        for row in tree.get_children():
            tree.delete(row)
        for index, row in df.iterrows():
            tree.insert('', 'end', iid=row["ID"], values=list(row))

    def search():
        term = search_var.get()
        result = Admin.search_records(term)
        load_treeview(result)

    def delete_selected():
        selected = tree.selection()
        if not selected:
            messagebox.showwarning("Select Record", "No record selected.")
            return
        for record_id in selected:
            Admin.delete_record(int(record_id))
        load_treeview(Admin.load_data())

    def edit_selected():
        selected = tree.selection()
        if not selected:
            messagebox.showwarning("Select Record", "No record selected.")
            return
        record_id = int(selected[0])
        df = Admin.load_data()
        record = df[df["ID"] == record_id].values[0]

        edit_win = Toplevel(dash)
        edit_win.title("Edit Admin")

        entries = []
        for i, field in enumerate(Admin.headers):
            tk.Label(edit_win, text=field).grid(row=i, column=0)
            entry = tk.Entry(edit_win, width=50)
            entry.insert(0, record[i])
            entry.grid(row=i, column=1)
            if field == "ID":
                entry.config(state='disabled')
            entries.append(entry)

        def save_changes():
            updated = [entry.get() for entry in entries]
            Admin.update_record(record_id, updated)
            edit_win.destroy()
            load_treeview(Admin.load_data())

        tk.Button(edit_win, text="Save", command=save_changes).grid(row=len(entries), column=1)

    def add_new():
        add_win = Toplevel(dash)
        add_win.title("Add New Admin")

        entries = []
        for i, field in enumerate(Admin.headers[1:]):
            tk.Label(add_win, text=field).grid(row=i, column=0)
            entry = tk.Entry(add_win, width=50)
            entry.grid(row=i, column=1)
            entries.append(entry)

        def save_new():
            new_data = [entry.get() for entry in entries]
            if any(val == "" for val in new_data):
                messagebox.showerror("Error", "Please fill all fields.")
                return
            Admin.add_record(new_data)
            add_win.destroy()
            load_treeview(Admin.load_data())

        tk.Button(add_win, text="Save", command=save_new).grid(row=len(entries), column=1)

    # GUI
    tk.Label(dash, text="Search:").pack(pady=5)
    tk.Entry(dash, textvariable=search_var).pack(pady=5)
    tk.Button(dash, text="Search", command=search).pack()

    columns = Admin.headers
    tree = ttk.Treeview(dash, columns=columns, show='headings')
    for col in columns:
        tree.heading(col, text=col)
        tree.column(col, width=150)
    tree.pack(pady=10, fill='both', expand=True)

    btn_frame = tk.Frame(dash)
    btn_frame.pack(pady=10)

    tk.Button(btn_frame, text="Add New Admin", command=add_new).grid(row=0, column=0, padx=10)
    tk.Button(btn_frame, text="Edit Selected", command=edit_selected).grid(row=0, column=1, padx=10)
    tk.Button(btn_frame, text="Delete Selected", command=delete_selected).grid(row=0, column=2, padx=10)
    tk.Button(btn_frame, text="Refresh", command=lambda: load_treeview(Admin.load_data())).grid(row=0, column=3, padx=10)

    load_treeview(Admin.load_data())

#Main Menu
def main_menu():
    root = tk.Tk()
    root.title("Hiking Registration & Admin App")
    root.geometry("400x300")

    tk.Label(root, text="Welcome to the Hiking Management System", font=("Arial", 14)).pack(pady=20)

    tk.Button(root, text="Open Admin Dashboard", width=30, command=open_admin_dashboard).pack(pady=10)

    # Placeholder for future integration
    tk.Button(root, text="Hiker Registration (Coming Soon)", width=30, state='disabled').pack(pady=10)

    root.mainloop()

# Start App 
if __name__ == "__main__":
    main_menu()


In [57]:
root = tk.Tk()
root.title("Hiker Registration Tracker")
root.geometry("550x750")

dark_mode = tk.BooleanVar(value=False)

def toggle_dark_mode():
    bg_color = "#2e2e2e" if dark_mode.get() else "SystemButtonFace"
    fg_color = "white" if dark_mode.get() else "black"
    
    root.configure(bg=bg_color)
    for widget in root.winfo_children():
        try:
            widget.configure(bg=bg_color, fg=fg_color)
        except:
            pass

#  Hiker Registration Section 
hiker_frame = tk.LabelFrame(root, text="🏞 Hiker Registration", padx=10, pady=10)
hiker_frame.pack(padx=10, pady=10, fill="both", expand=True)

tk.Label(hiker_frame, text="Name").pack()
name_entry = tk.Entry(hiker_frame)
name_entry.pack()

tk.Label(hiker_frame, text="Contact (Phone or Email)").pack()
contact_entry = tk.Entry(hiker_frame)
contact_entry.pack()

tk.Label(hiker_frame, text="Hike Name").pack()
hike_entry = tk.Entry(hiker_frame)
hike_entry.pack()

tk.Label(hiker_frame, text="Hike Date").pack()
hike_date_entry = DateEntry(hiker_frame, date_pattern='yyyy-mm-dd')
hike_date_entry.pack()

tk.Label(hiker_frame, text="Payment Status").pack()
payment_var = tk.StringVar()
tk.Radiobutton(hiker_frame, text="Full", variable=payment_var, value="Full").pack()
tk.Radiobutton(hiker_frame, text="Partial", variable=payment_var, value="Partial").pack()

tk.Label(hiker_frame, text="Gear Rental (Optional)").pack()
gear_options = ["Tent", "Sleeping Bag", "Hiking Pole"]
gear_vars = [tk.StringVar() for _ in gear_options]
for opt, var in zip(gear_options, gear_vars):
    tk.Checkbutton(hiker_frame, text=opt, variable=var, onvalue=opt, offvalue="").pack(anchor='w')

tk.Button(hiker_frame, text="Submit Registration", command=submit).pack(pady=10)

# ---------- Admin Access Section ----------
admin_frame = tk.LabelFrame(root, text="👤 Admin Access", padx=10, pady=10)
admin_frame.pack(padx=10, pady=10, fill="x")

tk.Button(admin_frame, text="Admin Login", command=admin_login).pack(pady=5)

root.mainloop()
